In [1]:
import os, sys
import numpy as np
import pandas as pd
import json

# from txyl_common.biocyc_facade.pgdb import Pgdb, Dat, Traceable
from local.caching import load, save

In [2]:
root = './cache/'
usage_data = []
for f in os.listdir(root):
    if not f.startswith('metabolite_usage'): continue
    with open(f'{root}/{f}') as j:
        usage_data += json.load(j)
    print(f'{f} {" "*25}', end='\r')

len(usage_data)

19999

In [4]:
fi, fo, fc = {}, {}, {}
for id, ins, outs, cats in usage_data:
    for ref, lst in zip([fi, fo, fc], [ins, outs, cats]):
        for c in lst:
            if c in ref: continue
            ref[c] = len(ref)

flens = len(fi), len(fo), len(fc)
flens

(14845, 16711, 1125)

In [25]:
features = {}
i = 0
for pre, ref in zip("i, o, c".split(', '), [fi, fo, fc]):
    for k, _ in ref.items():
        features[f"{pre}_{k}"] = i
        i += 1
len(features)

32681

In [8]:
xlen, ylen = len(usage_data), len(features)
ids = []
mat = np.ndarray(shape=(xlen, ylen), dtype=np.ushort)
for row_i, (id, ins, outs, cats) in enumerate(usage_data):
    ids.append(id)
    for pre, usage in zip("i, o, c".split(', '), [ins, outs, cats]):
        for k, c in usage.items():
            key = f"{pre}_{k}"
            i = features[key]
            mat[row_i, i] = c

mat

array([[ 2, 23,  5, ...,  0,  0,  0],
       [ 2, 35,  6, ...,  0,  0,  0],
       [ 2, 32,  5, ...,  0,  0,  0],
       ...,
       [ 4, 31,  6, ...,  0,  0,  0],
       [ 2,  9,  4, ...,  0,  0,  0],
       [ 2, 25,  1, ...,  0,  0,  0]], dtype=uint16)

In [26]:
flst = ['']*len(features)
for k, i in features.items():
    flst[i] = k

In [28]:
df = pd.DataFrame(mat, index=ids, columns=flst, dtype=np.ushort)

In [31]:
save('biocyc_metabolite_usage', df)

compressing & caching data to [{WORKSPACE}/main/test_models/cache/biocyc_metabolite_usage.pkl.gz]


In [27]:
flst[-5:]

['c_UDP-GLUCURONATE',
 'c_CPD-12575',
 'c_NOREPINEPHRINE',
 'c_Nucleoside-Monophosphates',
 'c_3-4-DIHYDROXYBENZOATE']

In [6]:
# root = "../../data/txyl_local/biocyc/pgdbs"
# pfs = os.listdir(root)
# total = len(pfs)
# for i, pf in enumerate(pfs):
#     print(f"{i+1} of {total}: {pf} {' '*50}", end='\r')
#     db = Pgdb(f"{root}/{pf}")
    
#     def get_ins_outs(v: dict):
#         direction: str = v.get('REACTION-DIRECTION', ['unk'])[0]
#         lefts = set(v.get('LEFT', []))
#         rights = set(v.get('RIGHT', []))

#         left_o = lefts.difference(rights)
#         right_o = rights.difference(lefts)
#         catalysts = lefts.intersection(rights)
        
#         if 'LEFT' not in direction or 'RIGHT' not in direction: # reversible
#             all = lefts.union(rights)
#             return all, all, catalysts
#         else:
#             li = direction.index('LEFT')
#             ri = direction.index('RIGHT')

#             if li < ri:
#                 return left_o, right_o, catalysts
#             else:
#                 return right_o, left_o, catalysts

#     consumption, production, catalyst_use = {}, {}, {}
#     def addc(d: dict, v: str):
#         d[v] = d.get(v, 0)+1

#     for k, v in db.GetDataTable(Dat.REACTIONS).items():
#         ins, outs, cats = get_ins_outs(v)
#         for lst, ref in zip([ins, outs, cats], [consumption, production, catalyst_use]):
#             for mol in lst:
#                 addc(ref, mol)

#     return 


In [11]:
# def get_ins_outs(v: dict):
#     direction: str = v.get('REACTION-DIRECTION', ['unk'])[0]
#     lefts = set(v.get('LEFT', []))
#     rights = set(v.get('RIGHT', []))

#     left_o = lefts.difference(rights)
#     right_o = rights.difference(lefts)
#     catalysts = lefts.intersection(rights)
    
#     if 'LEFT' not in direction or 'RIGHT' not in direction: # reversible
#         all = lefts.union(rights)
#         return all, all, catalysts
#     else:
#         li = direction.index('LEFT')
#         ri = direction.index('RIGHT')

#         if li < ri:
#             return left_o, right_o, catalysts
#         else:
#             return right_o, left_o, catalysts

# consumption, production, catalyst_use = {}, {}, {}
# def addc(d: dict, v: str):
#     d[v] = d.get(v, 0)+1

# for k, v in db.GetDataTable(Dat.REACTIONS).items():
#     ins, outs, cats = get_ins_outs(v)
#     for lst, ref in zip([ins, outs, cats], [consumption, production, catalyst_use]):
#         for mol in lst:
#             addc(ref, mol)

In [4]:
# assign_methods = set()

# for k, v in db.GetDataTable(Dat.ENZYMES).items():
#     ak = "BASIS-FOR-ASSIGNMENT"
#     assert ak in v
#     assign_methods.add(v[ak][0])
#     print(v)
#     break
    
# assign_methods

{'TYPES': ['Enzymatic-Reactions'], 'COMMON-NAME': ['acetyl-CoA carboxylase carboxyltransferase subunit alpha/beta'], 'BASIS-FOR-ASSIGNMENT': [':AUTOMATED-NAME-MATCH'], 'ENZYME': ['DZ96_RS0232580-MONOMER'], 'INSTANCE-NAME-TEMPLATE': ['ENZRXN-*'], 'PHYSIOLOGICALLY-RELEVANT?': ['T'], 'REACTION': ['ACETYL-COA-CARBOXYLTRANSFER-RXN']}


{':AUTOMATED-NAME-MATCH'}

In [5]:
# BL = {':INFERRED-TRANSPORT-RXN', ':PATHWAY-HOLE-FILLER'}